In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# import ipynb
# from ipynb.fs.all.HelperFunctions import *
%run HelperFunction.ipynb

This code loads your CSV file, splits the data into a training set and a test set, and creates a DataLoader for each. The DataLoader can be used to iterate through the data in batches, which is useful for training a neural network.

You can replace 'yourfile.csv' with the path to your actual file. Also, note that this assumes your CSV file doesn't have a header. If it does, you might need to skip the first row.

In [2]:
class JFSKAccelDataset(Dataset):
    def __init__(self, data, labels, sequence_length=10):
        self.data = [data[i:i+sequence_length] for i in range(len(data) - sequence_length + 1)]
        self.labels = labels[(int)(sequence_length/2) - 1 : len(data) - (sequence_length - (int)(sequence_length/2))]
        # change to get the majority
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx].transpose(0, 1), self.labels[idx]  # Transposing the sequence and channel dimensions


In [3]:
class JFLoader:
    def __init__(self, file_path, sequence_length = 10):
        self.sequence_length = sequence_length

        # Gather file info
        # self.file_directory, self.beginning_descriptors, self.file_name, self.ending_descriptors, self.file_extension, self.specifier_values = SKFileNameHandler.read_file_name(file_path)
        self.file_directory, _, _, _, self.file_extension, self.specifier_values = SKFileNameHandler.read_file_name(file_path)

        # Load CSV file
        # WORKAREA_PATH = './'
        # class_num = 8 ## Although annoying, requiring this be manually adjusted
        #     ## to open the file also means that a later instance of this number will be correct
        # prev input: WORKAREA_PATH + f'Data/COMBINED_Type5-WithClassNum{class_num}-Freq10-Labeled_Motion-sessions_23-24_Fall.csv'

        if self.file_extension == ".csv":
            dataframe = pd.read_csv(file_path)
        else:
            raise NotImplementedError("JFLoader is only equipped to open .csv files.")
        
        # code test file: Data/Week 1/Left then Right/Processed/Type3-Freq10-Labeled_Motion-sessions_2023-08-26_17-25-54.csv
        # classifier training file: Data/COMBINED_Type3-Freq10-Labeled_Motion-sessions_23-24_Fall.csv

        # Get data and labels from dataframe
        input_num = SKDescriptors.NUM_OF_INPUTS_PER_TYPE[self.specifier_values["Type"]]
        data = dataframe.iloc[:, :input_num].values  # x, y, z data
        labels = dataframe.iloc[:, input_num:].values  # labels


        # sequence_length = 10

        # Split data into training and test sets
        data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        # Convert data to tensors
        data_train = torch.tensor(data_train, dtype=torch.float32)  
        data_test = torch.tensor(data_test, dtype=torch.float32)

        # Convert labels to tensors and get max index (assuming one-hot encoding)
        labels_train = torch.argmax(torch.tensor(labels_train, dtype=torch.float32), dim=1)
        labels_test = torch.argmax(torch.tensor(labels_test, dtype=torch.float32), dim=1)

        # Create data loaders
        train_dataset = JFSKAccelDataset(data_train, labels_train, self.sequence_length)
        test_dataset = JFSKAccelDataset(data_test, labels_test, self.sequence_length)

        self.train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        self.test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
